In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [3]:
pip freeze > model_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
data_train_df = pd.read_csv("data_train_csv")
data_smauto_train_df = pd.read_csv("data_smauto_train_csv")
data_smin_train_df = pd.read_csv("data_smin_train_csv")

data_test_df = pd.read_csv("data_test_csv")
data_val_df = pd.read_csv("data_val_csv")

In [5]:
#own module which takes in df for train, test and val and returns in x and y format.  Can also sample.
from data_formatting import data_format

In [6]:
X_train, X_test, X_val, y_train, y_test, y_val = data_format(data_smin_train_df, data_test_df, data_val_df, None, None)

Highly correlated data - could remove some of the features



In [7]:
# Assuming you have X_train, X_test, y_train, and y_test

# Autoencoder
input_dim = X_train.shape[1]

for encoding_dim in [2, 4, 8, 16, 32]:
    print(encoding_dim)
    # Encoder
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    encoder_layer = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_layer)
    encoder = tf.keras.models.Model(inputs=input_layer, outputs=encoder_layer)

    # Decoder
    decoder_layer = tf.keras.layers.Dense(input_dim, activation='sigmoid')(encoder_layer)
    autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder_layer)

    # Compile the model with binary crossentropy since it's a multi-class classification problem
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    # Train the Autoencoder
    autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_data=(X_val, X_val))

    # Extract features using the trained encoder
    X_train_encoded = encoder.predict(X_train)
    X_test_encoded = encoder.predict(X_test)

    # Use the encoded features with a Random Forest model
    rf_classifier = RandomForestClassifier(random_state=7)
    rf_classifier.fit(X_train_encoded, y_train)

    # Make predictions on the test set
    y_pred_test = rf_classifier.predict(X_test_encoded)

    # Evaluate the performance
    test_accuracy = accuracy_score(y_test, y_pred_test)
    print(f"Random Forest Accuracy: {test_accuracy}")


2
Epoch 1/100
9002/9002 [==============================] - 9s 1ms/step - loss: -467852.4688 - val_loss: -1351597.3750
Epoch 2/100
9002/9002 [==============================] - 9s 1ms/step - loss: -2679447.5000 - val_loss: -4602409.5000
Epoch 3/100
9002/9002 [==============================] - 9s 988us/step - loss: -6594999.0000 - val_loss: -9618968.0000
Epoch 4/100
9002/9002 [==============================] - 9s 1ms/step - loss: -12172822.0000 - val_loss: -16378386.0000
Epoch 5/100
9002/9002 [==============================] - 9s 993us/step - loss: -19387272.0000 - val_loss: -24862786.0000
Epoch 6/100
9002/9002 [==============================] - 9s 979us/step - loss: -28246572.0000 - val_loss: -35071532.0000
Epoch 7/100
9002/9002 [==============================] - 9s 964us/step - loss: -38720820.0000 - val_loss: -46991820.0000
Epoch 8/100
9002/9002 [==============================] - 9s 979us/step - loss: -50813092.0000 - val_loss: -60613376.0000
Epoch 9/100
9002/9002 [====================

In [ ]:
X_train, X_test, X_val, y_train, y_test, y_val = data_format(data_smin_train_df, data_test_df, data_val_df, None, None)